In [ ]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from psycopg.conninfo import make_conninfo

import os
import psycopg

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings()

connectionString = make_conninfo(
  host=os.getenv('DB_HOST'),
  port=os.getenv('DB_PORT'),
  dbname=os.getenv('DB_DATABASE'),
  user=os.getenv('DB_USER'),
  password=os.getenv('DB_PASSWORD')
)

In [9]:

# Load sample documents into the database

directory = './data'

# Loop over all files in the directory, create embeddings, save them to the database
for file_name in os.listdir(directory):
  file_path = os.path.join(directory, file_name)
  with open(file_path, 'r') as file:
      content = file.read()
      embedding_results = embeddings.embed_query(content)
      with psycopg.connect(connectionString) as conn:
        insert_query = "INSERT INTO documents (file_name, content, embedding) VALUES (%s, %s, %s)"
        insert_data = (file_name, content, embedding_results)
        conn.execute(insert_query, insert_data)
        conn.commit()


In [10]:
search_text = "How has music changed over the years?"

In [11]:
# embed search text
embedding_results = embeddings.embed_query(search_text)

In [15]:
# find documents based on cosin similarity to the embedded search text
with psycopg.connect(connectionString) as conn: 
  select_query = "SELECT match_documents(%s,%s,%s)"
  query_data = (f'{embedding_results}', 0.8, 10)
  results = conn.execute(select_query, query_data).fetchall()
  for result in results:
    print(result)  

(('7', 'Music has been a part of human culture for thousands of years, with various forms of music emerging from different regions and cultures. Music can evoke emotions, tell stories, and bring people together. From classical music to hip-hop to country, there is a genre of music to suit every taste.', 'music_and_culture.txt', '0.8553450551592263'),)
(('9', 'Musical instruments are an essential part of creating music. They range from simple instruments like the recorder or harmonica to complex instruments like the piano or guitar. Learning to play a musical instrument can provide numerous benefits, including improving cognitive function, reducing stress, and boosting self-esteem.', 'music_and_instruments.txt', '0.8048350573666982'),)
